In [1]:
import pandas as pd

DATA_PATH = "LoLesports_data/"

train = pd.read_csv(f"{DATA_PATH}teams_train.csv")
test = pd.read_csv(f"{DATA_PATH}teams_test.csv")

train.shape, test.shape

((9913, 111), (2324, 111))

In [2]:
games_per_patch = train.groupby("patch").size() / 2

ban_list = ["ban1", "ban2", "ban3", "ban4", "ban5"]
agg_list = {ban: "value_counts" for ban in ban_list}

tmp = train.groupby("patch").agg(agg_list).fillna(0)
ban_sum = tmp[ban_list].sum(axis=1)

ban_rate = (ban_sum / games_per_patch).sort_index()
ban_rate = (ban_rate * 100).round(2)

df = pd.DataFrame({
    'ban_count': ban_sum,
    'ban_rate': ban_rate,
})

df.head()

ban_count  ban_rate
patch                              
12.01 Akali          32.0     28.83
      Akshan          7.0      6.31
      Aphelios        9.0      8.11
      Azir            4.0      3.60
      Braum           4.0      3.60

In [4]:
pick_list = ["pick1", "pick2", "pick3", "pick4", "pick5"]
agg_list = {pick: "value_counts" for pick in pick_list}

tmp = train.groupby("patch").agg(agg_list).fillna(0)
pick_sum = tmp[pick_list].sum(axis=1)

pick_rate = (pick_sum / games_per_patch).sort_index()
pick_rate = (pick_rate * 100).round(2)

ban_pick_count = ban_sum + pick_sum
ban_pick_rate = (ban_pick_count / games_per_patch).sort_index()
ban_pick_rate = (ban_pick_rate * 100).round(2)

tmp = pd.DataFrame({
    "pick_count": pick_sum,
    "pick_rate": pick_rate,
    "ban_pick_count": ban_pick_count,
    "ban_pick_rate": ban_pick_rate,
})

df = pd.concat([df, tmp], axis=1)
df.fillna(0, inplace=True)
df.head()

ban_count  ban_rate  pick_count  pick_rate  ban_pick_count  \
patch                                                                        
12.01 Akali          32.0     28.83        23.0      20.72            55.0   
      Akshan          7.0      6.31         5.0       4.50            12.0   
      Aphelios        9.0      8.11        51.0      45.95            60.0   
      Azir            4.0      3.60         3.0       2.70             7.0   
      Braum           4.0      3.60         4.0       3.60             8.0   

                ban_pick_rate  
patch                          
12.01 Akali             49.55  
      Akshan            10.81  
      Aphelios          54.05  
      Azir               6.31  
      Braum              7.21

In [5]:
melted = train.melt(id_vars=['patch', 'result'], value_vars=pick_list, var_name='pick_order', value_name='champion')
tmp = melted.groupby(['patch', 'champion']).agg(
    wins=('result', 'sum'),
    games=('result', 'count'),
    win_rate=("result", "mean")
)

df = pd.concat([df, tmp], axis=1)
df.head()

ban_count  ban_rate  pick_count  pick_rate  ban_pick_count  \
patch                                                                        
12.01 Akali          32.0     28.83        23.0      20.72            55.0   
      Akshan          7.0      6.31         5.0       4.50            12.0   
      Aphelios        9.0      8.11        51.0      45.95            60.0   
      Azir            4.0      3.60         3.0       2.70             7.0   
      Braum           4.0      3.60         4.0       3.60             8.0   

                ban_pick_rate  wins  games  win_rate  
patch                                                 
12.01 Akali             49.55  12.0   23.0  0.521739  
      Akshan            10.81   2.0    5.0  0.400000  
      Aphelios          54.05  23.0   51.0  0.450980  
      Azir               6.31   1.0    3.0  0.333333  
      Braum              7.21   2.0    4.0  0.500000

In [6]:
df.fillna(0, inplace=True)
df.reset_index(inplace=True)
df.to_csv("output/champions_stat.csv", index=False)